## **Section by Hana – ETL Process: Olist Brazilian E-commerce Dataset**

___
The Olist dataset contains multiple **relational CSV files** representing Brazil’s largest e-commerce platform. Each file captures a different aspect of the order lifecycle—customers, sellers, products, payments, and reviews.

### Main Files:
- `orders`: Order-level data with status and timestamps  
- `order_items`: Product-level info per order  
- `products`: Product metadata  
- `payments`: Payment methods and values  
- `reviews`: Customer feedback  
- `customers`: Customer location  
- `sellers`: Seller details  
- `geo`: Geolocation of postal codes  
- `category translation`: Product categories in English

### Data Timeframe Selection
- The original dataset spans from 2016-09-04 21:15:19 to 2018-10-17 17:30:18. However, for the purpose of this project, only a slice covering the most recent three months — from 2018-07-17 to 2018-10-17 — has been used for analysis

### Why Relational?
Tables are linked via keys like `order_id`, `product_id`, and `customer_id`, making it ideal for multi-table analysis.

This structure supports a clean **ETL process**:  
→ **Extract** raw files  
→ **Transform** and join data  
→ **Load** for analysis or modeling
___

## 1. **Extraction**

In [273]:
import pandas as pd

# Correct relative paths from notebooks/ to data/raw/
orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
order_items = pd.read_csv("../data/raw/olist_order_items_dataset.csv")
products = pd.read_csv("../data/raw/olist_products_dataset.csv")
payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")
customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")
sellers = pd.read_csv("../data/raw/olist_sellers_dataset.csv")
geo = pd.read_csv("../data/raw/olist_geolocation_dataset.csv")
categories = pd.read_csv("../data/raw/product_category_name_translation.csv")


#### *A. Initial Data Inspection*

In [274]:
datasets = {
    "Orders": orders,
    "Order Items": order_items,
    "Products": products,
    "Payments": payments,
    "Reviews": reviews,
    "Customers": customers,
    "Sellers": sellers,
    "Geolocation": geo,
    "Category Translation": categories
}

for name, df in datasets.items():
    print(f"\n{name} Head:")
    print(df.head())



Orders Head:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00    

In [275]:
orders['order_purchase_timestamp'].max()
   

'2018-10-17 17:30:18'

In [276]:
orders['order_purchase_timestamp'].min() 

'2016-09-04 21:15:19'

In [277]:
datasets = {
    "Orders": orders,
    "Order Items": order_items,
    "Products": products,
    "Payments": payments,
    "Reviews": reviews,
    "Customers": customers,
    "Sellers": sellers,
    "Geolocation": geo,
    "Category Translation": categories
}
for name, df in datasets.items():
    print(f"\n{name} Info:")
    print(df.info())


Orders Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

Order Items Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_i

___
### *B. Initial Inspection Observations*

From the initial `.info()` checks across the Olist datasets, we observe the following:

- **Data Volume**: Most datasets contain tens of thousands of entries, with `geolocation` being the largest (~1 million rows) and `category translation` the smallest (71 rows).
- **Data Types**: Many columns are stored as `object` type, particularly those representing IDs and dates. This may require type conversion (e.g., to `datetime`) during preprocessing.
- **Missing Data**:
  - The `orders` dataset has some missing timestamps in delivery-related fields.
  - The `products` dataset has a few missing values in product attributes.
  - The `reviews` dataset has many missing values in comment titles and messages, which may be optional fields.
- **Relational Structure**: 
  - Shared keys like `order_id`, `product_id`, `customer_id`, and `seller_id` across tables suggest strong relational integrity.
  - This supports potential joins to create enriched views for further analysis.

These findings guide us to perform data cleaning and transformations in the upcoming ETL steps.
____


## **2. Transformaiton**

### *A. Conversion of date columns to `datetime` type for purpose of filtering*

___
Identify all datetime-related columns across all DataFrames.
___

In [278]:
# List all your DataFrames in a dictionary
dfs = {
    "Orders": orders,
    "Order Items": order_items,
    "Products": products,
    "Payments": payments,
    "Reviews": reviews,
    "Customers": customers,
    "Sellers": sellers,
    "Geolocation": geo,
    "Category Translation": categories
}

# Loop through and check for datetime-like columns
for name, df in dfs.items():
    date_cols = [col for col in df.columns if 'date' in col.lower() or 'timestamp' in col.lower()]
    if date_cols:
        print(f"{name} has date-related columns: {date_cols}")


Orders has date-related columns: ['order_purchase_timestamp', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
Order Items has date-related columns: ['shipping_limit_date']
Reviews has date-related columns: ['review_creation_date', 'review_answer_timestamp']


___
Convert relevant date-related columns from string (object) to datetime64[ns] format for proper time-based analysis and filtering.
____
```python

In [279]:
# Orders table
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

# Order Items table
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'])

# Reviews table
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])


___
Verify / Check the data types of each DataFrame to ensure successful conversion using .dtypes.
____
```python


In [280]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                        object
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [281]:
order_items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [282]:
reviews.dtypes

review_id                          object
order_id                           object
review_score                        int64
review_comment_title               object
review_comment_message             object
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object

### *B. Extracting Latest 3-Month Data Snapshot from Orders and Related Tables*

In [283]:
# Step 1: Define latest 3-month window
latest_date = orders['order_purchase_timestamp'].max()
three_months_ago = latest_date - pd.DateOffset(months=3)

# Step 2: Filter orders
filtered_orders = orders[
    orders['order_purchase_timestamp'] >= three_months_ago
].copy()

# Step 3: Get list of order_ids
filtered_order_ids = filtered_orders['order_id'].unique()

# Step 4: Filter related tables using order_id
filtered_order_items = order_items[
    order_items['order_id'].isin(filtered_order_ids)
].copy()

filtered_payments = payments[
    payments['order_id'].isin(filtered_order_ids)
].copy()

filtered_reviews = reviews[
    reviews['order_id'].isin(filtered_order_ids)
].copy()

# Step 5: Filter customers using customer_id from filtered_orders
filtered_customers = customers[
    customers['customer_id'].isin(filtered_orders['customer_id'])
].copy()

# Step 6: Filter products using product_id from filtered_order_items
filtered_products = products[
    products['product_id'].isin(filtered_order_items['product_id'])
].copy()

# Step 7: Filter sellers using seller_id from filtered_order_items
filtered_sellers = sellers[
    sellers['seller_id'].isin(filtered_order_items['seller_id'])
].copy()

# Step 8: Filter category translation using product_category_name from filtered_products
filtered_categories = categories[
    categories['product_category_name'].isin(filtered_products['product_category_name'])
].copy()

# Step 9: Filter geolocation using both customer and seller zip codes
customer_zip_codes = filtered_customers['customer_zip_code_prefix'].unique()
seller_zip_codes = filtered_sellers['seller_zip_code_prefix'].unique()
all_zip_codes = pd.concat([
    pd.Series(customer_zip_codes),
    pd.Series(seller_zip_codes)
]).unique()

filtered_geo = geo[
    geo['geolocation_zip_code_prefix'].isin(all_zip_codes)
].copy()

___
- To focus the analysis on the most recent three months of activity, the datasets were filtered based on the order_purchase_timestamp field. Orders falling within this time window were retained, and all other tables were subsequently filtered to include only the relevant records linked to these recent orders.

- This filtering approach ensures temporal consistency across all datasets. It also preserves the relational integrity necessary for downstream merging, analysis, and visualization. Redundant or unrelated entries were excluded, allowing for a cleaner, more focused subset of the data that reflects the latest consumer behavior and transactions
____
```python

### *C.Merging All Filtered Data Based on Recent Orders (Last 3 Months)*


In [ ]:
# Start from filtered_orders to keep all recent orders
df = filtered_orders.copy()

# Merge with order_items (may create NaNs if no items for some orders indicating maay be incomplete or canceled orders)
df = df.merge(filtered_order_items, on='order_id', how='left')

# Merge with payments and reviews (still left join to retain all orders)
df = df.merge(filtered_payments, on='order_id', how='left')
df = df.merge(filtered_reviews, on='order_id', how='left')

# Merge with customers using customer_id
df = df.merge(filtered_customers, on='customer_id', how='left')

# Merge with products and sellers (based on order_items info)
df = df.merge(filtered_products, on='product_id', how='left')
df = df.merge(filtered_sellers, on='seller_id', how='left')

# Merge with product category translation
df = df.merge(filtered_categories, on='product_category_name', how='left')

# --------- Optional: Add geolocation info ---------

# 1. Customer geolocation: average lat/lng by zip
geo_avg_customer = filtered_geo.groupby('geolocation_zip_code_prefix')[
    ['geolocation_lat', 'geolocation_lng']
].mean().reset_index()

geo_avg_customer.rename(columns={
    'geolocation_zip_code_prefix': 'customer_zip_code_prefix',
    'geolocation_lat': 'customer_lat',
    'geolocation_lng': 'customer_lng'
}, inplace=True)

df = df.merge(geo_avg_customer, on='customer_zip_code_prefix', how='left')

# 2. Seller geolocation: average lat/lng by zip
geo_avg_seller = filtered_geo.groupby('geolocation_zip_code_prefix')[
    ['geolocation_lat', 'geolocation_lng']
].mean().reset_index()

geo_avg_seller.rename(columns={
    'geolocation_zip_code_prefix': 'seller_zip_code_prefix',
    'geolocation_lat': 'seller_lat',
    'geolocation_lng': 'seller_lng'
}, inplace=True)

df = df.merge(geo_avg_seller, on='seller_zip_code_prefix', how='left')


In [285]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11777 entries, 0 to 11776
Data columns (total 44 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       11777 non-null  object        
 1   customer_id                    11777 non-null  object        
 2   order_status                   11777 non-null  object        
 3   order_purchase_timestamp       11777 non-null  datetime64[ns]
 4   order_approved_at              11704 non-null  object        
 5   order_delivered_carrier_date   11584 non-null  datetime64[ns]
 6   order_delivered_customer_date  11499 non-null  datetime64[ns]
 7   order_estimated_delivery_date  11777 non-null  datetime64[ns]
 8   order_item_id                  11684 non-null  float64       
 9   product_id                     11684 non-null  object        
 10  seller_id                      11684 non-null  object        
 11  shipping_limit_

In [286]:
print("Final Filtered and Merged df:")
print("Earliest:", df['order_purchase_timestamp'].min())
print("Latest:", df['order_purchase_timestamp'].max())

Final Filtered and Merged df:
Earliest: 2018-07-17 17:32:09
Latest: 2018-10-17 17:30:18


___
- After filtering the datasets to retain only the relevant records from the most recent three months, multiple data tables were merged to form a single, comprehensive DataFrame.

- This merging process was performed using key identifiers such as order_id, customer_id, product_id, and seller_id to preserve the relational structure between orders, customers, products, payments, reviews, sellers, and geolocation data.

- As a result, a unified dataset with 11,777 entries and 44 columns was created, containing detailed transaction-level information suitable for in-depth analysis. Temporal coverage in this final dataset ranges from 2018-07-17 to 2018-10-17. Missing values in some columns reflect real-world data gaps such as unreviewed orders or incomplete delivery records.
___

### *D. Check for duplicates*

In [287]:
df.duplicated().sum()

0

___
- No duplicate entries were found in the final merged DataFrame, ensuring data integrity for analysis.
____
```python

### *E. Check for Missing Values*

In [288]:
# Percentage of missing values
missing_percent = (df.isnull().mean() * 100).round(2)
missing_summary = pd.DataFrame({
    'Missing Values': df.isnull().sum(),
    'Percentage': missing_percent
})
missing_summary = missing_summary[missing_summary['Missing Values'] > 0].sort_values(by='Missing Values', ascending=False)

missing_summary

,Missing Values,Percentage
review_comment_title,7418,62.99
review_comment_message,7166,60.85
order_delivered_customer_date,278,2.36
order_delivered_carrier_date,193,1.64
product_category_name_english,173,1.47
product_category_name,156,1.32
product_photos_qty,156,1.32
product_description_lenght,156,1.32
product_name_lenght,156,1.32
seller_lat,104,0.88


In [289]:
# Remove columns with more than 50% missing values
threshold = 0.5  # 50%
df = df.loc[:, df.isnull().mean() <= threshold]

#### Fill datetime columns with a related datetime column (if known), else use forward fill

In [290]:
import warnings

# Suppress all warnings 
warnings.filterwarnings('ignore')

# Fill datetime columns with a related datetime column (if known), else use forward fill
datetime_cols = df.select_dtypes(include='datetime64[ns]').columns

for col in datetime_cols:
    if col == 'order_approved_at':
        df[col] = df[col].fillna(df['order_purchase_timestamp'])  # domain-specific logic
    else:
        df[col] = df[col].fillna(method='ffill')  # forward fill as default for datetime

#### Fill missing numeric values with mean or median using skewness check

In [291]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

for col in numeric_cols:
    if df[col].isnull().sum() > 0:  # only process columns with missing values
        skewness = df[col].skew()
        if abs(skewness) > 1:  # high skew: use median
            df[col] = df[col].fillna(df[col].median())
        else:  # low or moderate skew: use mean
            df[col] = df[col].fillna(df[col].mean())

#### Fill object (categorical) columns with mode

In [292]:
# Fill object (categorical) columns with mode
object_cols = df.select_dtypes(include='object').columns

for col in object_cols:
    if df[col].isnull().any():
        mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else 'unknown'
        df[col] = df[col].fillna(mode_value)


#### Verify/Check the missing values after filling

In [293]:
df.isnull().sum()


order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
order_item_id                    0
product_id                       0
seller_id                        0
shipping_limit_date              0
price                            0
freight_value                    0
payment_sequential               0
payment_type                     0
payment_installments             0
payment_value                    0
review_id                        0
review_score                     0
review_creation_date             0
review_answer_timestamp          0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
product_category_name            0
product_name_lenght              0
product_description_

In [294]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11777 entries, 0 to 11776
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       11777 non-null  object        
 1   customer_id                    11777 non-null  object        
 2   order_status                   11777 non-null  object        
 3   order_purchase_timestamp       11777 non-null  datetime64[ns]
 4   order_approved_at              11777 non-null  object        
 5   order_delivered_carrier_date   11777 non-null  datetime64[ns]
 6   order_delivered_customer_date  11777 non-null  datetime64[ns]
 7   order_estimated_delivery_date  11777 non-null  datetime64[ns]
 8   order_item_id                  11777 non-null  float64       
 9   product_id                     11777 non-null  object        
 10  seller_id                      11777 non-null  object        
 11  shipping_limit_

### *F. Add Calculated Fields*

#### Profit Margin

In [295]:
df['profit_margin'] =round((df['price'] - df['freight_value']) / df['price'],2)
df['profit_margin'].head()

0    0.81
1    0.88
2    0.36
3    0.84
4    0.95
Name: profit_margin, dtype: float64

___
- **Profit Margin** calculated using the formula:

  **Profit Margin** = (Price − Freight Value) / Price


  - This metric shows how much of the **product price** is left after subtracting the **freight cost** (logistics).
  - A **higher value** indicates more profit relative to the selling price.

  #### 🔍 Example Interpretation:
  - For the **first row**, `profit_margin = 0.81`:
    - This means that **81% of the product price is retained as profit** after covering shipping costs.
    - Only **19% of the price** went toward freight expenses.
___

#### Purchase Frequency per Customer

In [296]:
purchase_freq = df.groupby('customer_unique_id')['order_id'].nunique()
df['purchase_frequency'] = df['customer_unique_id'].map(purchase_freq)

___
To understand **how often each customer makes a purchase**, **Purchase frequency** based on unique order IDs associated with each customer was calculated.

- Helps identify **repeat customers** vs. **one-time buyers**
- Enables **customer segmentation** based on loyalty or shopping behavior
- Useful for **churn prediction**, **marketing strategies**, and **customer lifetime value** analysis
___


### *G. Handle Outliers* 

In [297]:
import numpy as np

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)        # 25th percentile (lower quartile)
    Q3 = df[col].quantile(0.75)        # 75th percentile (upper quartile)
    IQR = Q3 - Q1                      # Interquartile Range
    lower = Q1 - 1.5 * IQR             # Lower bound for outlier detection
    upper = Q3 + 1.5 * IQR             # Upper bound for outlier detection

    # Cap values above upper to upper, and below lower to lower
    df[col] = np.where(df[col] > upper, upper,
                       np.where(df[col] < lower, lower, df[col]))


___
##### Handling Outliers Using the IQR Method
This code handles outliers in all numeric columns using the **IQR (Interquartile Range)** method by **capping** them:

- Outliers **below the lower bound** are replaced with the **lower bound value**
- Outliers **above the upper bound** are replaced with the **upper bound value**
- Values **within bounds** are left unchanged


##### Why It's Used
- Reduces noise in the dataset without removing rows  
- Keeps extreme values from skewing models or visualizations  
- Maintains data size while improving quality
____
```python


### *H. Standardize Formats & Remove Noise*

##### Ensuring consistent formats across all object (string) columns

In [298]:
object_cols = df.select_dtypes(include='object').columns

for col in object_cols:
    df[col] = df[col].astype(str).str.lower().str.strip()

___
This code cleans all object (text) columns by applying three transformations:

- **`astype(str)`**:  
  Ensures the column is treated as a string (even if some values are not).

- **`.str.lower()`**:  
  Converts all text to **lowercase**.  
  Example: `'Delivered'` → `'delivered'`  
  Useful for consistency during comparison, grouping, etc.

- **`.str.strip()`**:  
  Removes **leading and trailing whitespace**.  
  Example: `' shipped '` → `'shipped'`  
  Prevents issues where `'shipped '` ≠ `'shipped'`.

##### Why It’s Important

- Makes data **uniform and consistent**
- Prevents errors in **filtering, grouping, or merging**
- A good practice in **data cleaning** for all text columns
___

##### Datatype Conversion

In [299]:
df['order_item_id'] = df['order_item_id'].astype(str)
df['seller_zip_code_prefix'] = df['seller_zip_code_prefix'].astype(str)

In [300]:
columns_to_int = [
    'review_score',
    'product_name_lenght',
    'product_description_lenght',
    'product_photos_qty',
]

# Convert using astype after handling any potential NaN or float values
for col in columns_to_int:
    df[col] = df[col].round().astype('Int64')  # 'Int64' allows for missing values

#### Verify/Check the data types of each DataFrame to ensure successful conversion.

In [301]:
print(df['order_item_id'].dtype)
print(df['seller_zip_code_prefix'].dtype)

object
object


In [302]:
# view selected columns' data types
columns_to_int = [
    'review_score',
    'product_name_lenght',
    'product_description_lenght',
    'product_photos_qty',
]

print(df[columns_to_int].dtypes)

review_score                  Int64
product_name_lenght           Int64
product_description_lenght    Int64
product_photos_qty            Int64
dtype: object
